In [159]:
import pandas as pd

# Cargar el archivo
df_norespondieron = pd.read_excel(r"C:\Users\El economista\Documents\GitHub\-BigDataUBA-Grupo-25\TPs\TP4\df_completo_original.xlsx")

In [160]:
import pandas as pd
# Crear la nueva columna "edad²" con los valores al cuadrado
# Renombrar la columna "edad" a "edad2"
df_norespondieron.rename(columns={"ch06": "edad2"}, inplace=True)
df_norespondieron["edad²"] = df_norespondieron["edad2"] ** 2
df_norespondieron["pp3e_tot"] = df_norespondieron["pp3e_tot"].fillna(0)
df_norespondieron["pp3f_tot"] = df_norespondieron["pp3f_tot"].fillna(0)
df_norespondieron["horastrab"] = df_norespondieron["pp3e_tot"] + df_norespondieron["pp3f_tot"]
# Aplicar el coeficiente de ajuste
df_norespondieron["p21"] = df_norespondieron["p21"].astype(float)  # Convertir a float antes de modificar
df_norespondieron.loc[df_norespondieron["ano4"] == 2004, "p21"] *= 314.621568
# Calcular el salario semanal
df_norespondieron["salario_semanal"] = df_norespondieron["p21"] / 40
# Función para calcular años de educación
def calcular_educ(row):
    if row["ch12"] == 2:  # Primario
        return row["ch14"]
    elif row["ch12"] == 4:  # Secundario
        return 12 if row["ch13"] == 1 else row["ch14"]
    elif row["ch12"] == 6:  # Terciario
        return 15 if row["ch13"] == 1 else 12 + row["ch14"]
    elif row["ch12"] == 7:  # Universitario
        return 17 if row["ch13"] == 1 else 12 + row["ch14"]
    elif row["ch12"] == 9:  # Educación especial
        return None  # No comparable
    else:
        return None  # Ns/Nr u otro valor

# Aplicar la función al DataFrame
df_norespondieron["educ"] = df_norespondieron.apply(calcular_educ, axis=1)
# Crear variable mujer
df_norespondieron["mujer"] = (df_norespondieron["ch04"] == 2).astype(int) 
df_train = df_norespondieron[df_norespondieron["estado"].isin([1, 2])].copy()  # Filtrar solo ocupados y desocupados
df_norespondieron["norespondieron"] = (df_norespondieron["estado"] == 0).astype(int) 

In [161]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Si en el entrenamiento ya tenés X_train definido, usá ese conjunto; 
# de lo contrario, ajustamos con X_norespondieron o alguna otra partición.
scaler.fit(X_norespondieron)

StandardScaler()

In [162]:
import numpy as np
X_norespondieron = df_norespondieron[columnas_features].copy()
X_norespondieron = X_norespondieron.copy()  # Asegura que trabajamos sobre una copia real del DataFrame
X_norespondieron["educ"] = X_norespondieron["educ"].fillna(0)  # Reemplaza NaN por 0
X_norespondieron_scaled = scaler.transform(X_norespondieron)


In [163]:
import statsmodels.api as sm

modelo = sm.OLS(y_train, X_train).fit()  # Ajustar modelo de regresión
coeficientes = modelo.params  # Extraer coeficientes
print(coeficientes)  # Verificar valores

edad2             0.001846
educ              0.001248
mujer             0.056568
horastrab        -0.001039
norespondieron    0.000000
dtype: float64


In [164]:
print("Columnas en X_train:", X_train.columns)
print("Columnas en X_test:", X_test.columns)

Columnas en X_train: Index(['edad2', 'educ', 'mujer', 'horastrab', 'norespondieron'], dtype='object')
Columnas en X_test: Index(['const', 'edad2', 'educ', 'mujer', 'horastrab', 'norespondieron'], dtype='object')


In [165]:
print(f"Columnas en X_train: {X_train.shape[1]}")
print(f"Columnas en X_test: {X_test.shape[1]}")
print(f"Dimensión coeficientes: {coeficientes.shape}")

Columnas en X_train: 5
Columnas en X_test: 6
Dimensión coeficientes: (5,)


In [166]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Filtrar ocupados y desocupados en la base de testeo
df_test = df_norespondieron[df_norespondieron["estado"].isin([1, 2])].copy()

# Asegurar que las columnas coincidan con el entrenamiento
X_test = df_test[columnas_features].copy()

# Convertir datos y eliminar valores NaN
X_test = np.nan_to_num(X_test)
X_test = pd.DataFrame(X_test, columns=columnas_features)  # Mantener nombres de columnas

# **Eliminar columnas duplicadas en X_train**
X_train = X_train.loc[:, ~X_train.columns.duplicated()]

# **Eliminar "const" duplicada si ya existe en X_train y X_test**
if 'const' in X_train.columns:
    X_train = X_train.drop(columns=['const'])
if 'const' in X_test.columns:
    X_test = X_test.drop(columns=['const'])

# **Asegurar que X_test tenga las mismas columnas que X_train**
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# **Agregar constante correctamente**
X_train = sm.add_constant(X_train, has_constant='add')
X_test = sm.add_constant(X_test, has_constant='add')

# **Verificar dimensiones antes de la predicción**
print(f"Columnas en X_train: {X_train.shape[1]}")
print(f"Columnas en X_test: {X_test.shape[1]}")
print(f"Dimensión coeficientes: {coeficientes.shape}")

# **Multiplicación matricial segura**
if X_test.shape[1] == coeficientes.shape[0]:
    salario_predicho = np.dot(X_test.values, coeficientes)  # Multiplicación segura
else:
    print("Error: Las dimensiones de X_test y coeficientes no coinciden. Verifica los datos.")

# **Asignar valores predichos al DataFrame**
df_test["salario_semanal_predicho"] = salario_predicho

# **Mostrar resumen de valores predichos**
print(df_test[["salario_semanal_predicho"]].describe())

Columnas en X_train: 6
Columnas en X_test: 6
Dimensión coeficientes: (5,)
Error: Las dimensiones de X_test y coeficientes no coinciden. Verifica los datos.
       salario_semanal_predicho
count              18113.000000
mean                   0.073297
std                    0.065952
min                   -1.957980
25%                    0.037641
50%                    0.074501
75%                    0.110530
max                    0.296452


In [167]:
import pandas as pd
import numpy as np
# 1. Definir las columnas features (sin incluir la variable objetivo 'desocupado')
columnas_features = ["edad2", "educ", "mujer", "horastrab", "norespondieron"]
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train = df_train[columnas_features]
X_train.loc[:, "educ"] = X_train["educ"].fillna(0)
X_norespondieron.loc[:, "educ"] = X_norespondieron["educ"].fillna(0)
y_train = df_train["estado"].map({1: 0, 2: 1})  # Ocupado → 0, Desocupado → 1

KeyError: "['norespondieron'] not in index"

In [168]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Valores reales de salario en la base de testeo
y_test = df_test["salario_semanal"]

# Calcular MSE
mse_test = mean_squared_error(y_test, salario_predicho)

# Calcular RMSE (Raíz del MSE)
rmse_test = np.sqrt(mse_test)

# Calcular MAE (Error Absoluto Medio)
mae_test = mean_absolute_error(y_test, salario_predicho)

# Mostrar resultados
print(f"MSE test: {mse_test:.4f}")
print(f"RMSE test: {rmse_test:.4f}")
print(f"MAE test: {mae_test:.4f}")

MSE test: 67448245.9736
RMSE test: 8212.6881
MAE test: 4728.3698


In [158]:
import numpy as np

print(np.isinf(X_test).sum())  # Muestra cuántos valores infinitos hay en X_test
print(np.isinf(y_test).sum())  # Muestra cuántos valores infinitos hay en y_test

const             0
edad2             0
educ              0
mujer             0
horastrab         0
norespondieron    0
dtype: int64
0
